In [ ]:
from twi_ksvd.omp import OMP
import numpy as np

## Test and usage of OMP

### Syntethic data

In [ ]:
p = 10
K = 100
N = 100
x = np.ones(p)
X = np.random.random((p, N))
D = np.random.random((p, K))

### Test

In [ ]:
alpha = OMP(x, D, 3)

np.linalg.norm(x - D @ alpha)

## Test and usage of COSTW

In [ ]:
from twi_ksvd.costw import COSTW

t1 = np.arange(20)
t2 = np.arange(10)


x1 = np.cos(t1)
y1 = np.cos(2*t2)

costw, delta = COSTW(x1,y1)

In [ ]:
(delta @ np.ones(delta.shape[1]) == 1).all()

In [ ]:
(delta @ y1).shape == x1.shape

## Test and usage of TWI-OMP

In [ ]:
from twi_ksvd.omp import TWI_OMP

D_list = [D[:,j] for j in range(D.shape[1])]

alpha, deltas = TWI_OMP(x, D_list, 3)

In [ ]:
len(deltas)

In [ ]:

approx = np.zeros_like(x)

for j in range(len(D_list)):
    if alpha[j] != 0:
        approx += alpha[j] * deltas[j] @ D_list[j]

np.linalg.norm(x - approx)

## Test and usage of kSVD

In [ ]:
from twi_ksvd.ksvd import kSVD
model = kSVD( 10)
tau = 3
model. fit(X,D,tau )

# Test and Usage of TWI-kSVD

In [1]:
import numpy as np
from twi_ksvd.ksvd import TWI_kSVD
model = TWI_kSVD( 10)
X = np.random.random((100,4))
D = np.random.random((100,10)).T
tau = 3
model. fit(X,D,tau )

  0%|          | 0/4 [00:00<?, ?it/s]

iteration number : 0, eps : inf, delta_eps : inf
     Step 1 | Compute Sparse Codes


  0%|          | 0/4 [00:00<?, ?it/s]

(4, 100)
     Step 2 | Update Dictionnary
iteration number : 1, eps : 1062.9720353947273, delta_eps : inf
     Step 1 | Compute Sparse Codes


  0%|          | 0/4 [00:00<?, ?it/s]

(4, 100)
     Step 2 | Update Dictionnary
iteration number : 2, eps : 1057.4809866154033, delta_eps : 5.491048779324046
     Step 1 | Compute Sparse Codes


  0%|          | 0/4 [00:00<?, ?it/s]

(4, 100)
     Step 2 | Update Dictionnary
iteration number : 3, eps : 1058.5157995841953, delta_eps : 1.034812968792039
     Step 1 | Compute Sparse Codes


 25%|██▌       | 1/4 [00:04<00:12,  4.01s/it]


KeyboardInterrupt: 